In [2]:
import pandas as pd
import numpy as np
import sys

#sys.path.append(r'C:\Program Files\DIgSILENT\PowerFactory 2024\Python\3.11')
sys.path.append(r'D:\STUFF\PowerFactory\Python\3.11')
if __name__ == "__main__":
    import powerfactory as pf
app = pf.GetApplication()
if app is None:
    sys.path.append(r'D:\STUFF\PowerFactory\Python\3.11')
    if __name__ == "__main__":
        import powerfactory as pf
    app = pf.GetApplication()
    if app is None:
        raise Exception('getting Powerfactory application failed')

In [3]:
#define project name and study case    
projName = 'BESS GFL-GFM_r03(3)'
study_case = '00_Comparison.IntCase'

#activate project
project = app.ActivateProject(projName)
proj = app.GetActiveProject()

#get the study case folder and activate project
oFolder_studycase = app.GetProjectFolder('study')
oCase = oFolder_studycase.GetContents(study_case)[0]
oCase.Activate()

#get load flow object and execute
oLoadflow=app.GetFromStudyCase('ComLdf') #get load flow object
oLoadflow.Execute() #execute load flow

0

In [4]:
#get the generators and their active/reactive power and loading
Generators = app.GetCalcRelevantObjects('*.ElmSym')
for gen in Generators: #loop through list
    name = getattr(gen, 'loc_name') # get name of the generator
    actPower = getattr(gen,'c:p') #get active power
    reacPower = getattr(gen,'c:q') #get reactive power
    genloading = getattr(gen,'c:loading') #get loading
    #print results
    print('%s: P = %.2f MW, Q = %.2f MVAr, loading = %.0f percent' %(name,actPower,reacPower,genloading))

print('-----------------------------------------')

#get the lines and print their loading
Lines=app.GetCalcRelevantObjects('*.ElmLne')
for line in Lines: #loop through list
    name = getattr(line, 'loc_name') # get name of the line
    value = getattr(line, 'c:loading') #get value for the loading
    #print results
    #print('Loading of the line: %s = %.2f percent' %(name,value))

print('-----------------------------------------')

#get the buses and print their voltage
Buses=app.GetCalcRelevantObjects('*.ElmTerm')
for bus in Buses: #loop through list
    name = getattr(bus, 'loc_name') # get name of the bus
    amp = getattr(bus, 'm:u1') #get voltage magnitude
    phase = getattr(bus, 'm:phiu') #get voltage angle
    #print results
    print('Voltage at %s = %.2f pu %.2f deg' %(name,amp,phase))

print('-----------------------------------------')

Loads=app.GetCalcRelevantObjects('*.ElmLod')
for load in Loads: #loop through list
    name = getattr(load, 'loc_name') # get name of the bus
    power = getattr(load, 'plini')
    #print results
    print('Active Power at %s = %.3f MW' %(name,power))

Diesel Generator 1: P = 3.80 MW, Q = 0.00 MVAr, loading = 40 percent
Diesel Generator 2: P = 1.50 MW, Q = 0.00 MVAr, loading = 22 percent
Diesel Generator 1: P = 3.80 MW, Q = 0.00 MVAr, loading = 80 percent
Diesel Generator 2: P = 1.50 MW, Q = 0.00 MVAr, loading = 35 percent
-----------------------------------------
-----------------------------------------
Voltage at T0 = 1.00 pu 0.00 deg
Voltage at T1 = 0.98 pu 148.02 deg
Voltage at T10 = 0.99 pu 148.69 deg
Voltage at T11 = 0.99 pu 148.70 deg
Voltage at T2 = 0.98 pu 148.32 deg
Voltage at T3 = 0.99 pu 148.79 deg
Voltage at T4 = 0.99 pu 148.73 deg
Voltage at T5 = 0.99 pu 148.67 deg
Voltage at T6 = 0.99 pu 148.63 deg
Voltage at T7 = 0.99 pu 148.63 deg
Voltage at T8 = 0.99 pu 148.67 deg
Voltage at T9 = 0.99 pu 148.67 deg
Voltage at Term Conv 1 = 1.00 pu 0.00 deg
Voltage at Term Conv 2 = 1.00 pu -0.74 deg
Voltage at T0 = 1.00 pu 0.00 deg
Voltage at T1 = 0.97 pu 146.63 deg
Voltage at T10 = 0.99 pu 147.26 deg
Voltage at T11 = 0.99 pu 147.27

#### function declaration

In [5]:
def createDf():
    columns = []
    Buses=app.GetCalcRelevantObjects('*.ElmTerm')
    i = 0
    for bus in Buses: #loop through list
        i += 1
        columns.append(getattr(bus, 'loc_name') + "Umag")
        columns.append(getattr(bus, 'loc_name') + "Uang")
        columns.append(getattr(bus, 'loc_name') + "Pact")
        columns.append(getattr(bus, 'loc_name') + "Prea")
    print(columns)
    tup = (columns, [])
    df = pd.DataFrame(tup)
    df.columns = columns
    df = df.drop([0,1])
    return df

def measure(dfResults):
    dfResults.loc[len(dfResults)] = pd.Series(dtype='float64')
    Buses=app.GetCalcRelevantObjects('*.ElmTerm')
    for bus in Buses: #loop through list
        dfResults.at[dfResults.shape[0]-1, getattr(bus, 'loc_name') + "Umag"] = getattr(bus, 'm:ui')#measure Umag here
        
        dfResults.at[dfResults.shape[0]-1, getattr(bus, 'loc_name') + "Uang"] = getattr(bus, 'm:phiu')#measure Uang here
        
        dfResults.at[dfResults.shape[0]-1, getattr(bus, 'loc_name') + "Pact"] = getattr(bus, 'm:Pflow')#measure Pact here
        
        dfResults.at[dfResults.shape[0]-1, getattr(bus, 'loc_name') + "Prea"] = getattr(bus, 'm:Qflow')#measure Prea here

def splitter(dfResults, text):
    dfResults.loc[len(dfResults)] = pd.Series(dtype='float64')
    dfResults.at[dfResults.shape[0]-1,0] = text


#### setup (get vars and restore initial values)

In [6]:
#initial setup
Generators = app.GetCalcRelevantObjects('*.ElmSym')
for gen in Generators: #loop through list
    match getattr(gen, 'loc_name'):
        case 'Diesel Generator 1':
            DGen1 = gen
        case 'Diesel Generator 2':
            DGen2 = gen

Buses=app.GetCalcRelevantObjects('*.ElmTerm')
for bus in Buses: #loop through list
    match getattr(bus, 'loc_name'):
        case 'T7':
            Bus7 = bus
        case 'T8':
            Bus8 = bus


Loads=app.GetCalcRelevantObjects('*.ElmLod')
for load in Loads: #loop through list   
    match getattr(load, 'loc_name'):
        case 'Load_3':
            Load3 = load
        case 'Load_4':
            Load4 = load
        case 'Load_7':
            Load7 = load
        case 'Load_8':
            Load8 = load

setattr(DGen1, 'outserv', 0)
setattr(Load3, 'plini', 0.55)
setattr(Load4, 'plini', 0.445)    
setattr(Load7, 'plini', 0.605)  
setattr(Load8, 'plini', 0.675)  
dfResults = createDf()
 



['T0Umag', 'T0Uang', 'T0Pact', 'T0Prea', 'T1Umag', 'T1Uang', 'T1Pact', 'T1Prea', 'T10Umag', 'T10Uang', 'T10Pact', 'T10Prea', 'T11Umag', 'T11Uang', 'T11Pact', 'T11Prea', 'T2Umag', 'T2Uang', 'T2Pact', 'T2Prea', 'T3Umag', 'T3Uang', 'T3Pact', 'T3Prea', 'T4Umag', 'T4Uang', 'T4Pact', 'T4Prea', 'T5Umag', 'T5Uang', 'T5Pact', 'T5Prea', 'T6Umag', 'T6Uang', 'T6Pact', 'T6Prea', 'T7Umag', 'T7Uang', 'T7Pact', 'T7Prea', 'T8Umag', 'T8Uang', 'T8Pact', 'T8Prea', 'T9Umag', 'T9Uang', 'T9Pact', 'T9Prea', 'Term Conv 1Umag', 'Term Conv 1Uang', 'Term Conv 1Pact', 'Term Conv 1Prea', 'Term Conv 2Umag', 'Term Conv 2Uang', 'Term Conv 2Pact', 'Term Conv 2Prea', 'T0Umag', 'T0Uang', 'T0Pact', 'T0Prea', 'T1Umag', 'T1Uang', 'T1Pact', 'T1Prea', 'T10Umag', 'T10Uang', 'T10Pact', 'T10Prea', 'T11Umag', 'T11Uang', 'T11Pact', 'T11Prea', 'T2Umag', 'T2Uang', 'T2Pact', 'T2Prea', 'T3Umag', 'T3Uang', 'T3Pact', 'T3Prea', 'T4Umag', 'T4Uang', 'T4Pact', 'T4Prea', 'T5Umag', 'T5Uang', 'T5Pact', 'T5Prea', 'T6Umag', 'T6Uang', 'T6Pact', '

## Aufgabe 1

In [7]:
oLoadflow=app.GetFromStudyCase('ComLdf') #get load flow object
oLoadflow.Execute() #execute load flow
measure(dfResults) 
splitter(dfResults,"Task 1 finsihed")

## Aufgabe 2

In [8]:
for i in range(1,11):
    setattr(Load3, 'plini', 0.55 + i*0.0045)
    
    oLoadflow=app.GetFromStudyCase('ComLdf') #get load flow object
    oLoadflow.Execute() #execute load flow
    measure(dfResults) 
    #ERGEBNISSE HIER AUSLESEN
    
    setattr(Load4, 'plini', 0.445 + i*0.0045)  

    oLoadflow.Execute() #execute load flow
    measure(dfResults) 
    #ERGEBNISSE HIER AUSLESEN

splitter(dfResults,"Task 1 finsihed")
#Switch off Diesel Generator, Test in Power Factory
#conditions: Load 3 active Power to 0.595/load4 active power to 0.49
splitter(dfResults, "Task 2, load 3 and 4 at max"  )

setattr(DGen1, 'outserv', 1)

for i in range(1,6):
    setattr(Load7, 'plini', 0.605 + i*0.0045)
    
    oLoadflow.Execute() #execute load flow
    measure(dfResults) 
    #ERGEBNISSE HIER AUSLESEN
    
    setattr(Load8, 'plini', 0.675 + i*0.0045)  

    oLoadflow.Execute() #execute load flow
    measure(dfResults) 
    #ERGEBNISSE HIER AUSLESEN


In [9]:
dfResults.to_excel("results.xlsx")

In [10]:
Generators = app.GetCalcRelevantObjects('*.ElmSym')
i = 0
for gen in Generators: #loop through list
    i = i+1
    name = getattr(gen, 'loc_name') # get name of the generator
    actPower = getattr(gen,'c:p') #get active power
    reacPower = getattr(gen,'c:q') #get reactive power
    genloading = getattr(gen,'c:loading') #get loading
    #print results
    print('%s: P = %.2f MW, Q = %.2f MVAr, loading = %.0f percent' %(name,actPower,reacPower,genloading))

    #setattr(gen, 'loc_name', "Diesel Generator " + str(i)) # get name of the generator
    name = getattr(gen, 'loc_name') # get name of the generator
    #print results
    print('%s: P = %.2f MW, Q = %.2f MVAr, loading = %.0f percent' %(name,actPower,reacPower,genloading))

Diesel Generator 1: P = 3.80 MW, Q = 0.00 MVAr, loading = 40 percent
Diesel Generator 1: P = 3.80 MW, Q = 0.00 MVAr, loading = 40 percent
Diesel Generator 2: P = 1.50 MW, Q = 0.00 MVAr, loading = 22 percent
Diesel Generator 2: P = 1.50 MW, Q = 0.00 MVAr, loading = 22 percent


AttributeError: 'DataObject' object has no attribute 'c:p'

In [11]:
# calculate initial conditions
oInit = app.GetFromStudyCase('ComInc') #get initial condition calculation object
oInit.Execute()

#run RMS-simulation
oRms = app.GetFromStudyCase('ComSim') #get RMS-simulation object
oRms.Execute()

0

In [13]:
import numpy as np
import pandas as pd

def getResults():    
    #get result file
    elmRes = app.GetFromStudyCase('*.ElmRes')    
    app.ResLoadData(elmRes)
  
    #Get number of rows and columns
    NrRow = app.ResGetValueCount(elmRes,0)
    
    #get objects of interest
    oSDG1 = app.GetCalcRelevantObjects('Diesel Generator 1.ElmSym')[0]
    oSDG2 = app.GetCalcRelevantObjects('Diesel Generator 2.ElmSym')[0]
    oBus1 = app.GetCalcRelevantObjects('T1.ElmTerm')[0]
    oLine3_4 = app.GetCalcRelevantObjects('L_3_4.ElmLne')[0]

    #Get index of variable of interest
    ColIndex_time = app.ResGetIndex(elmRes,elmRes,'b:tnow')
    ColIndex_ut = app.ResGetIndex(elmRes,oSDG1,'s:ut')
    ColIndex_P = app.ResGetIndex(elmRes,oSDG1,'s:P1')
    ColIndex_Q = app.ResGetIndex(elmRes,oSDG1,'s:Q1')
    ColIndex_speed = app.ResGetIndex(elmRes,oSDG1,'s:xspeed')
    ColIndex_u_bus1 = app.ResGetIndex(elmRes,oBus1,'m:u')
    ColIndex_loading_line_3_4 = app.ResGetIndex(elmRes,oLine3_4,'c:loading')
    
    #pre-allocate result variables
    result_time = np.zeros((NrRow,))
    result_ut = np.zeros((NrRow))
    result_P = np.zeros((NrRow))
    result_Q = np.zeros((NrRow))    
    result_speed = np.zeros((NrRow))    
    result_u_bus1 = np.zeros((NrRow))    
    result_loading_line_3_4 = np.zeros((NrRow))    
    
    #get results for each time step
    for i in range(NrRow):    
        result_time[i] = app.ResGetData(elmRes,i,ColIndex_time)[1]
        result_ut[i] = app.ResGetData(elmRes,i,ColIndex_ut)[1]
        result_P[i] = app.ResGetData(elmRes,i,ColIndex_P)[1]
        result_Q[i] = app.ResGetData(elmRes,i,ColIndex_Q)[1]       
        result_speed[i] = app.ResGetData(elmRes,i,ColIndex_speed)[1]       
        result_u_bus1[i] = app.ResGetData(elmRes,i,ColIndex_u_bus1)[1]       
        result_loading_line_3_4[i] = app.ResGetData(elmRes,i,ColIndex_loading_line_3_4)[1]       
    
    results = pd.DataFrame()
    results['time'] = result_time
    results['P'] = result_P
    results['Q'] = result_Q
    results['ut'] = result_ut
    results['speed'] = result_speed
    results['u_bus1'] = result_u_bus1
    results['loading_line_3_4'] = result_loading_line_3_4
    return results

#query results
RES = getResults()

In [14]:
RES[0:20]

,time,P,Q,ut,speed,u_bus1,loading_line_3_4
0,-1.0000,-1.0000,-1.0000,1.0,1.0,-1.0000,-1.0000
1,-0.9990,-0.9990,-0.9990,1.0,1.0,-0.9990,-0.9990
2,-0.9980,-0.9980,-0.9980,1.0,1.0,-0.9980,-0.9980
3,-0.9970,-0.9970,-0.9970,1.0,1.0,-0.9970,-0.9970
4,-0.9960,-0.9960,-0.9960,1.0,1.0,-0.9960,-0.9960
5,-0.9950,-0.9950,-0.9950,1.0,1.0,-0.9950,-0.9950
6,-0.9940,-0.9940,-0.9940,1.0,1.0,-0.9940,-0.9940
7,-0.9930,-0.9930,-0.9930,1.0,1.0,-0.9930,-0.9930
8,-0.9920,-0.9920,-0.9920,1.0,1.0,-0.9920,-0.9920
9,-0.9910,-0.9910,-0.9910,1.0,1.0,-0.9910,-0.9910
